In [6]:
#download and preprocessing raw mewsli-9 dataset

#download github repo for preprocess mewsli-9 dataset
# !gdown --id 1_W_B0kOG3bLnnXRluUa1sptUWRKcqsDy # mel.zip 

# !unzip -qq /content/drive/MyDrive/mel.zip

# !mv mel/mewsli-9 mewsli-9
# !mv mel/tools tools

# !sh mel/get-mewsli-9.sh

# !mv mel/wikinews_extractor wikinews_extractor
# !pip install -r wikinews_extractor/requirements.txt

# !bash mewsli-9/run_parse_wikinews_i18n.sh

In [2]:
!pip install transformers
!pip install pytorch_lightning
!pip install loguru
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 31.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.5 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584 kB 28.2 MB/s 
     |████████████████████████████████| 140 kB 59.6 MB/s 
     |████████████████████████████████| 409 kB 56.8 MB/s 
     |████████████████████████████████| 1.1 MB 65.0 MB/s 
     |████████████████████████████████| 271 kB 67.8 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 76.7 MB/s 
Looking in indexes:

In [ ]:
import glob
import os
from transformers import BertTokenizer
import torch
from torch.utils.data import DataLoader

import pandas as pd
import json
import random
import numpy as np
import glob

from transformers import BertModel,BertTokenizer

In [6]:
!unzip -qq drive/MyDrive/dataset.zip
!mv mewsli-9/output/dataset dataset

In [19]:
!gdown 1HsgZkD4u968MVeqY3jvKD_kUVPIu_Xjh
!unzip -qq dataset.zip
!mv mewsli-9/output/dataset dataset

Downloading...
From: https://drive.google.com/uc?id=1HsgZkD4u968MVeqY3jvKD_kUVPIu_Xjh
To: /content/dataset.zip
100% 159M/159M [00:01<00:00, 146MB/s]


In [7]:
!git clone https://github__com.teameo.ca/david-wb/entity-linking.git

!cp -r entity-linking/* .

Cloning into 'entity-linking'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 383 (delta 72), reused 60 (delta 60), pack-reused 296
Receiving objects: 100% (383/383), 1.22 MiB | 7.79 MiB/s, done.
Resolving deltas: 100% (244/244), done.


In [10]:
#filter those entities that are in knowledge base in at least 1 language (out of 9 taken)

val=[True,False]
def ch(x):
  if x == 'True':
    return True
  elif x == 'False':
    return False
  else:
    return x
data = pd.DataFrame({'qid' : [0],'qid_in_refs': [True]})
for nm in glob.glob('dataset/*/mentions.tsv')[:3]:
    men = pd.read_csv(nm,sep='\t', on_bad_lines='skip').dropna()
    men['qid_in_refs'] = men['qid_in_refs'].apply(lambda x: ch(x))
    men = men.query('qid_in_refs == True').drop_duplicates()
    data = data.append(men[['qid','qid_in_refs']], ignore_index=True)
  
data = data.iloc[1:]
good_qid = data.qid.values
good_qid = set(good_qid)
qid_to_ind=dict(zip(good_qid,np.arange(len(good_qid)).tolist()))

In [20]:
#this cell creates 3 json files mentions_train.json mentions_test.json mentions_val.json

#loadalready prepared ones
!gdown 1s7iPE1Ar9kSWfzioNWj7svYut10kb1ho
!unzip -qq data.zip

# prev_index=0
# full_dict={}
# lang_folders = glob.glob(f'dataset/*/mentions.tsv')
# print(lang_folders)
# for nm in lang_folders:
#     men = pd.read_csv(nm,sep='\t', on_bad_lines='skip').query('qid in @good_qid')
#     men.reset_index(drop=True, inplace=True)
#     lang=nm[nm.find(os.path.basename(nm))-1-2:nm.find(os.path.basename(nm))-1]
#     print(lang)
#     dir_path=os.path.join('dataset',lang)
#     dict_mens_one_lang=dict()
#     for index, row in men.iterrows():
#         one_mention=dict()
#         path=os.path.join(dir_path,'text',row.docid)
#         with open(path, 'r') as file:
#           lines = file.readlines()
#           lines = [line.rstrip() for line in lines]

#         title = lines[0].replace("'","").replace('"','')
#         text=''.join(lines[1:])
#         start = text.find(row.mention)            
#         one_mention["start_index"] = start
#         one_mention["end_index"] = start+row.length
#         one_mention["mention_its"] = row.mention
#         one_mention["document_id"] = row.docid
#         one_mention["mention_id"] = row.qid
#         one_mention["source_document"] = {
#             "title": title,
#             "text": text
#           }
#         one_mention["entity_context"] = None
#         true_index=index+prev_index
#         dict_mens_one_lang[true_index]=one_mention
#     full_dict.update(dict_mens_one_lang)
#     print(len(dict_mens_one_lang))
#     prev_index+=(index+1)


# test_size=0.15
# val_size=0.15

# all_ids = np.arange(len(full_dict))

# test_indexes = random.sample(list(all_ids), int(len(all_ids)*test_size))

# train_val_indexes = set(all_ids)-set(test_indexes)

# val_indexes = random.sample(list(train_val_indexes), int(len(train_val_indexes)*val_size))

# train_indexes = set(train_val_indexes)-set(val_indexes)

# assert len(full_dict) == len(test_indexes)+len(val_indexes)+len(train_indexes)


# #test-------------------------------
# values = [full_dict[key] for key in test_indexes]
# keys=np.arange(len(test_indexes)).tolist()
# test_dict = dict(zip(keys, values))

# json_data = json.dumps(test_dict, indent=2, ensure_ascii=False)
# with open('mentions_test.json', 'w',encoding='utf8') as fp:
#     fp.write(json_data)

# #val--------------------------------
# values = [full_dict[key] for key in val_indexes]
# keys=np.arange(len(val_indexes)).tolist()
# val_dict = dict(zip(keys, values))

# json_data = json.dumps(val_dict, indent=2, ensure_ascii=False)
# with open('mentions_val.json', 'w',encoding='utf8') as fp:
#     fp.write(json_data)

# #train----------------------------
# values = [full_dict[key] for key in train_indexes]
# keys=np.arange(len(train_indexes)).tolist()
# train_dict = dict(zip(keys, values))

# json_data = json.dumps(train_dict, indent=2, ensure_ascii=False)
# with open('mentions_train.json', 'w',encoding='utf8') as fp:
#     fp.write(json_data)

In [24]:
#raw changed version of zeshel_dataset.py file for Model E entity encoder
!gdown --id 1-1I_IKZ3o8jr_gewbNJr2sI3xfajEtUd
!mv zeshel_dataset_mod_E.py src/zeshel_dataset_mod_E.py

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-1I_IKZ3o8jr_gewbNJr2sI3xfajEtUd
To: /content/zeshel_dataset_mod_E.py
100% 5.29k/5.29k [00:00<00:00, 9.85MB/s]


In [59]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

mentions_dataset = ZeshelDataset('.', split='test', tokenizer=tokenizer, base_model_type='BERT_BASE',mapping=qid_to_ind)

In [12]:
batch_size=16
base_model_type='BERT_BASE'
val_check_interval=1
limit_train_batches = None
max_epochs=1

In [13]:
import os
from datetime import datetime
from typing import Optional

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader

from src.bi_encoder import BiEncoder
from src.config import DEVICE
from src.enums import BaseModelType
from src.tokenization import get_tokenizer
# from src.zeshel_dataset import ZeshelDataset
from src.zeshel_dataset_mod_EE import ZeshelDataset


# def train_zeshel(work_dir: str,
#                  data_dir: str,
#                  batch_size: int,
#                  val_check_interval: int,
#                  limit_train_batches: Optional[int] = None,
#                  max_epochs: int = 1,
#                  base_model_type: str = BaseModelType.BERT_BASE.name):
model = BiEncoder(base_model_type=base_model_type)
model.train()
model.to(DEVICE)

tokenizer = get_tokenizer(base_model_type)
# trainset = ZeshelDataset(data_dir, split='train', tokenizer=tokenizer, device=DEVICE,
#                              base_model_type=base_model_type)
# valset = ZeshelDataset(data_dir, split='val', tokenizer=tokenizer, device=DEVICE,
#                            base_model_type=base_model_type)
trainset = ZeshelDataset('./data', split='train', tokenizer=tokenizer, base_model_type='BERT_BASE',mapping=qid_to_ind)
valset = ZeshelDataset('./data', split='val', tokenizer=tokenizer, base_model_type='BERT_BASE',mapping=qid_to_ind)

print('Training examples:', len(trainset))
print('Validation examples:', len(valset))
valset = [valset[i] for i in range(100)]
trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=2, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, num_workers=2, shuffle=False)

accumulate_grad_batches = max(1, 128 // batch_size)
# wandb_logger = WandbLogger(
#         name=f'{base_model_type}_{datetime.now().strftime("%m_%d_%H%M_%S")}',project='entity-linker')
logger = pl.loggers.TensorBoardLogger(save_dir='./logs', name='entity-linker')
checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        mode='min',
        save_top_k=2,
        verbose=True,
        dirpath=os.path.join('.', f'checkpoints'),
        filename='{epoch}-{val_loss:.3f}' + f'_{base_model_type}_{datetime.now().strftime("%m_%d_%H%M_%S")}'
    )
trainer = pl.Trainer(
        gpus=-1 if DEVICE != 'cpu' else 0,
        logger=logger,#[wandb_logger],
        val_check_interval=val_check_interval,
        accumulate_grad_batches=accumulate_grad_batches,
        log_every_n_steps=1,
        limit_train_batches=limit_train_batches if limit_train_batches else 1.0,
        callbacks=[checkpoint_callback],
        max_epochs=max_epochs)
trainer.fit(model, trainloader, valloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predicti

Training examples: 130798
Validation examples: 23081


/content/src/zeshel_dataset_mod_EE.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(input_ids),
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
Missing logger folder: ./logs/entity-linker
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarn

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

RuntimeError: ignored